In [3]:
import fastbook
fastbook.setup_book()

In [4]:
from fastai.vision.all import * 
from fastbook import *
from fastai.vision.widgets import *


In [5]:

key = os.environ.get('AZURE_SEARCH_KEY', 'e6923970eafc46679e64c0b8582545b3')

In [6]:
flower_types = ['California sagebrush', 'Yarrow', 'Baby blue eyes flower', 'California aster', 'Sacred datura', 'bush poppy', 'bush Anemone', 'California buttercup flower', 'Chamise', 'deerweed', 'Old-mans beard flower', 'Tidy tips flower', 'Arroyo lupine', 'Coast Morning glory flower', 'Red Comlumbine', 'Beardtongues flower', 'Sticky monkey flower', 'showy penstemon', ' Mariposa lily', 'Anaphalis margaritacea', 'Gambelia speciosa', 'Apricot mallow']
path = Path('flowers')

In [7]:
if not path.exists():
    path.mkdir()
    for fl in flower_types:
        dest = (path/fl)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, fl)
        download_images(dest, urls=results.attrgot('contentUrl'))

In [8]:
fns = get_image_files(path)
failed = verify_images(fns)
failed

(#0) []

In [9]:
failed.map(Path.unlink)

(#0) []

In [10]:
class DataLoaders(GetAttr):
    def __init__(self, *loaders): self.loaders = loaders
    def __getitem__(self, i): return self.loaders[i]
    train,valid = add_props(lambda i,self: self[i])

In [11]:
flowers = DataBlock(
    blocks = (ImageBlock,CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct=0.2, seed=42),
    get_y = parent_label,
    item_tfms=Resize(128)
)


In [12]:
flowers = flowers.new(
    item_tfms=RandomResizedCrop(224, min_scale = 0.5),
    batch_tfms=aug_transforms()
)

dls = flowers.dataloaders(path)

In [13]:
learn = vision_learner(dls, resnet18, metrics = error_rate)
learn.fine_tune(4)

c:\Users\lauri\OneDrive\Documents\sillylilprojects\hackonthehill\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lauri\OneDrive\Documents\sillylilprojects\hackonthehill\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
learn.export()

In [16]:
path = Path()
path.ls(file_exts='.pkl')

(#1) [Path('export.pkl')]

In [21]:
learn_inf = load_learner(path/'export.pkl')
learn_inf.predict('images/aster.jpg')

('California aster',
 tensor(6),
 tensor([3.3518e-07, 1.0279e-06, 2.3955e-07, 1.1421e-05, 1.8100e-06, 1.5701e-09, 9.8817e-01, 7.3195e-05, 2.1865e-04, 5.6546e-05, 4.0814e-07, 1.7850e-07, 4.7245e-05, 3.6150e-06, 8.5645e-08, 1.9939e-07,
         1.1263e-02, 6.9505e-06, 1.3751e-04, 2.7362e-06, 1.7316e-06, 2.8026e-07]))

In [42]:
btn_upload = widgets.FileUpload()
img = PILImage.create(btn_upload.data[-1])

IndexError: list index out of range

In [39]:
lbl_pred = widgets.Label()
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl


Output()

In [40]:
pred,pred_idx,probs = learn_inf.predict(img)
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred
     

Label(value='Prediction: California aster; Probability: 0.9882')

In [37]:
btn_run = widgets.Button(description='Classify')
btn_run

Button(description='Classify', style=ButtonStyle())

In [34]:
def on_click_run(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred, pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
btn_run.on_click(on_click_run)


In [ ]:
btn_upload = widgets.FileUpload()

In [43]:
VBox([widgets.Label('Select flower!'), btn_upload, btn_run, out_pl, lbl_pred])